In [1]:
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

In [2]:
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()
default_bucket = sagemaker_session.default_bucket()

pipeline_name = f'pose-yolo-pipeline'
model_package_group_name = f'PoseYOLOv8'

In [3]:
print(sagemaker_session)

In [4]:
# make s3 folder named datasets-yolo-##
# upload datasets to s3

In [5]:
local_path = 'datasets-yolo/coco8-pose'
data_uri = f's3://{default_bucket}/{local_path}'
train_data_uri = f's3://{default_bucket}/{local_path}/train'
valid_data_uri = f's3://{default_bucket}/{local_path}/val'
test_data_uri = f's3://{default_bucket}/{local_path}/val'
print(data_uri)

s3://sagemaker-ap-northeast-2-600204363646/datasets-yolo/coco8-pose


In [6]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.parameters import ( 
    ParameterInteger, 
    ParameterString, 
    ParameterFloat
)

In [7]:
!git rev-parse --short=10 HEAD 

fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [8]:

#training_image_uri = f'600204363646.dkr.ecr.ap-northeast-2.amazonaws.com/yolov8-training-gpu:{tag}'
training_image_uri = f'600204363646.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-studio-d-zgsncn7muvbt:default-1675652336843'
model_path = f"s3://{default_bucket}/PoseTrain"

In [9]:
proceccing_instance_count = 1
training_instance_count = 1

training_data_input = ParameterString(name='TrainingData', default_value=train_data_uri)
valid_data_input = ParameterString(name='ValidData', default_value=valid_data_uri)
test_data_input = ParameterString(name='TestData', default_value=test_data_uri)

training_image = ParameterString(name='TrainingImage', default_value=training_image_uri)
training_instance_type = ParameterString(name='TrainingInstanceType', default_value='ml.p3.2xlarge') 
model_approval_status = ParameterString(name='ModelApprovalStatus', default_value='PendingManualApproval')
model_instance_type = ParameterString(name='ModelInstanceType', default_value='ml.m5.xlarge') 

map_threshold = ParameterFloat(name='mAPThreshold', default_value=0.2)

In [10]:
yolo_train = Estimator(
    image_uri=training_image,
    instance_type=training_instance_type,
    instance_count=training_instance_count,
    output_path=model_path,
    role=role,
    sagemaker_session=pipeline_session,
    volume_size=60,
)

In [11]:
yolo_train.set_hyperparameters(
    #model='yolov8n-pose',
    model='yolov8n-pose',
    experiment='posetest',
    epochs=30,
    batch=2,
    imgsz=640,
    seed=13,
)

In [12]:
train_args = yolo_train.fit(
    inputs={
        "train": TrainingInput(
            s3_data=training_data_input,
        ),
        "valid": TrainingInput(
            s3_data=valid_data_input,
        ),
        "test": TrainingInput(
            s3_data=test_data_input,
        ),
    }
)

/opt/conda/lib/python3.9/site-packages/sagemaker/workflow/pipeline_context.py:258: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [13]:
step_train = TrainingStep(
    name="PoseTrain",
    step_args=train_args,
)

In [14]:
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.processing import FrameworkProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.properties import PropertyFile

In [15]:
script_eval = FrameworkProcessor(
    estimator_cls=PyTorch,
    framework_version='1.12.1',
    py_version='py38',
    instance_type='ml.m5.xlarge',
    instance_count=1,
    base_job_name='script-pose-eval',
    role=role,
    sagemaker_session=pipeline_session,
)

In [16]:
eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination='/opt/ml/processing/model',
        ),
        ProcessingInput(source=test_data_input, destination="/opt/ml/processing/input")
    ],
    outputs=[
        ProcessingOutput(output_name='evaluation', source='/opt/ml/processing/evaluation'),
    ],
    source_dir='./eval/code',
    code='evaluate.py',
)

In [17]:
evaluation_report = PropertyFile(
    name='EvaluationReport', output_name='evaluation', path='evaluation.json'
)
step_eval = ProcessingStep(
    name='PoseEval',
    step_args=eval_args,
    property_files=[evaluation_report],
)

In [18]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.workflow.model_step import ModelStep
from sagemaker.model_metrics import MetricsSource, ModelMetrics

In [19]:
model = PyTorchModel(
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    framework_version='1.12',
    py_version='py38',
    sagemaker_session=pipeline_session,
    role=role,
    entry_point='inference.py',
    source_dir='./deploy/code',
    env={'TS_MAX_RESPONSE_SIZE':'20000000'},
)

In [20]:
# model_metrics = ModelMetrics(
#     model_statistics=MetricsSource(
#         s3_uri='{}/evaluation.json'.format(
#             step_eval.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']
#         ),
#         content_type='application/json',
#     )
# )

evaluation_s3_uri = "{}/evaluation.json".format(
    step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=evaluation_s3_uri,
        content_type="application/json",
    )
)

print(step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"])

INFO:sagemaker.processing:Uploaded ./eval/code to s3://sagemaker-ap-northeast-2-600204363646/script-pose-eval-2023-05-16-14-15-38-190/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-northeast-2-600204363646/script-pose-eval-2023-05-16-14-15-38-190/source/runproc.sh


s3://sagemaker-ap-northeast-2-600204363646/script-pose-eval-2023-05-16-14-15-37-769/output/evaluation


In [21]:
register_args = model.register(
    content_types=['image/jpeg'],
    response_types=['application/json'],
    inference_instances=['ml.p3.2xlarge', 'ml.g4dn.xlarge'],
    #transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)
step_register = ModelStep(name='PoseModel', step_args=register_args)

In [22]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep

In [23]:
step_create_model = ModelStep(
    name='PoseCreateModel',
    step_args=model.create(instance_type=model_instance_type),
)

In [24]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

In [25]:
cond_lte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="multiclass_classification_metrics.mAP50.value",
    ),
    right=map_threshold,
)

In [26]:
step_cond = ConditionStep(
    name="PoseMAPCondition",
    conditions=[
        cond_lte,
    ],
    if_steps=[
        step_register,
        #step_create_model,
    ],
    else_steps=[],
)

In [27]:
import json
from sagemaker.workflow.pipeline import Pipeline

In [28]:
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        training_data_input,
        valid_data_input,
        test_data_input,
        
        training_image,
        training_instance_type,
        model_approval_status,
        
        model_instance_type,
        map_threshold,
    ],
    steps=[
        step_train,
        step_eval,
        step_cond,
    ],
) 

In [29]:
definition = json.loads(pipeline.definition())
definition

INFO:sagemaker.processing:Uploaded ./eval/code to s3://sagemaker-ap-northeast-2-600204363646/pose-yolo-pipeline/code/27b7ee6e86192d952c8c177f6847fb7f/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-northeast-2-600204363646/pose-yolo-pipeline/code/042c27441c10cac7d270da211efc863b/runproc.sh


{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'TrainingData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-northeast-2-600204363646/datasets-yolo/coco8-pose/train'},
  {'Name': 'ValidData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-northeast-2-600204363646/datasets-yolo/coco8-pose/val'},
  {'Name': 'TestData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-northeast-2-600204363646/datasets-yolo/coco8-pose/val'},
  {'Name': 'TrainingImage',
   'Type': 'String',
   'DefaultValue': '600204363646.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-studio-d-zgsncn7muvbt:default-1675652336843'},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.p3.2xlarge'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'ModelInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'mAPThreshold', 'Type': 'Float', 'DefaultValue': 0

In [30]:
pipeline.upsert(role_arn=role)

INFO:sagemaker.processing:Uploaded ./eval/code to s3://sagemaker-ap-northeast-2-600204363646/pose-yolo-pipeline/code/27b7ee6e86192d952c8c177f6847fb7f/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-northeast-2-600204363646/pose-yolo-pipeline/code/042c27441c10cac7d270da211efc863b/runproc.sh


{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:600204363646:pipeline/pose-yolo-pipeline',
 'ResponseMetadata': {'RequestId': 'af945f8e-9d70-42b3-a276-fee6d1753106',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'af945f8e-9d70-42b3-a276-fee6d1753106',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '91',
   'date': 'Tue, 16 May 2023 14:15:40 GMT'},
  'RetryAttempts': 0}}

In [31]:
execution = pipeline.start()

In [32]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:600204363646:pipeline/pose-yolo-pipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:600204363646:pipeline/pose-yolo-pipeline/execution/2u3clwlbxlcg',
 'PipelineExecutionDisplayName': 'execution-1684246541274',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2023, 5, 16, 14, 15, 41, 203000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 5, 16, 14, 15, 41, 203000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:ap-northeast-2:600204363646:user-profile/d-zgsncn7muvbt/default-1675652336843',
  'UserProfileName': 'default-1675652336843',
  'DomainId': 'd-zgsncn7muvbt'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:ap-northeast-2:600204363646:user-profile/d-zgsncn7muvbt/default-1675652336843',
  'UserProfileName': 'default-1675652336843',
  'DomainId': 'd-zgsncn7muvbt'},
 'ResponseMetadata': {'RequestId': '290d2565-2f8e-4750-8e61-f0141c5dde7c',
 

/usr/bin/sh: 1: zip: not found


In [54]:
!sudo apt-get install zip unzip

/usr/bin/sh: 1: sudo: not found


In [52]:
!ls

bus.jpg		 deploy		      model.pt			   train
coco8-pose.yaml  docker-upload.ipynb  pose-yolov8-sagemaker.ipynb
datasets	 eval		      test-pose-model.ipynb
